## A) Import lib

In [1]:
from DataSet import DataSet
from SimGPBO_old import SimGPBO_old
from SimGPBO import SimGPBO
from PostProcessor import PostProcessor
import GPy
import numpy as np 
import matplotlib.pyplot as plt
import copy 
import torch 
import gpytorch
from GPcustom.models import GPytorchModel

In [2]:
def plot_map(ds: DataSet, vector: np.array, title: str=None) -> None:
    map = np.full((
                        np.max(ds.set['ch2xy'][:,0]),
                        np.max(ds.set['ch2xy'][:,1])
                        ), 
                        np.nan)
    for i in range(ds.set['ch2xy'].shape[0]):
        map[int(ds.set['ch2xy'][i,0]-1),int(ds.set['ch2xy'][i,1]-1)] = vector[i]
    plt.figure()
    plt.imshow(map, cmap='viridis', vmin=np.nanmin(map), vmax=np.nanmax(map))
    plt.colorbar(label="values")
    plt.title(title)
    plt.show()


## B) Import data

In [2]:
ds = DataSet('../data/','nhp','Cebus1_M1_190221.mat','cebus1')
ds.load_matlab_data() # load data from the dataset_file

## C) Simulation 

### C.1) simulation "OG" with gpytorch

In [3]:
sim = SimGPBO(name = 'test_og_', 
                    ds = copy.deepcopy(ds),
                    AF = 'UCB',
                    NB_REP = 1,
                    NB_IT = 10,
                    KAPPA = 6,
                    NB_RND = 1
                    )

sim.select_emgs([0])

sim.run_simulations(gp_origin='gpytorch', response_type='valid', 
                         hyperparams_storage=True, HP_estimation=True, manual_seed=True, outputscale=1., noise=0.01)

Global Progress: 100%|██████████| 1/1 [00:11<00:00, 11.85s/iter]

Elapsed time: 11.863202571868896 seconds
final save of test_og in:   results/gpbo_gpytorch_UCB_cebus1_test_og.npz


### C.2) simulation with gpytorch using QueriesInfo to predict (and to optimize the HPs)

In [3]:
sim = SimGPBO(name = 'test_pseudo_gpy_', 
                    ds = copy.deepcopy(ds),
                    AF = 'UCB',
                    NB_REP = 10,
                    NB_IT = 200,
                    KAPPA = 6,
                    NB_RND = 1
                    )

# sim.select_emgs([0])

sim.run_simulations(gp_origin='estimated_gpytorch', response_type='valid', 
                         hyperparams_storage=True, HP_estimation=False, manual_seed=True, 
                         outputscale=None, noise=None,
                         max_iters_training_gp=10)

Global Progress: 100%|██████████| 60/60 [17:12<00:00, 17.21s/iter]

Elapsed time: 1032.8427803516388 seconds
final save of test_pseudo_gpy_ in:   results/gpbo_estimated_gpytorch_UCB_cebus1_test_pseudo_gpy_.npz


### C.3) simulation with gpytorch using QueriesInfo to optimize the HPs

In [11]:
sim = SimGPBO(name = 'test_pseudo_hp_', 
                    ds = copy.deepcopy(ds),
                    AF = 'UCB',
                    NB_REP = 1,
                    NB_IT = 10,
                    KAPPA = 6,
                    NB_RND = 1
                    )

sim.select_emgs([0])

sim.run_simulations(gp_origin='estimated_hp_gpytorch', response_type='valid', 
                         hyperparams_storage=True, HP_estimation=False, manual_seed=True, outputscale=1., noise=0.01)

Global Progress: 100%|██████████| 1/1 [00:05<00:00,  5.64s/iter]

Elapsed time: 5.646507263183594 seconds
final save of test_pseudo_hp in:   results/gpbo_estimated_hp_gpytorch_UCB_cebus1_test_pseudo_hp.npz
